In [1]:
import json
import pandas as pd
import ipywidgets as widgets
import cufflinks as cf
from IPython.display import display, HTML
from plotly.graph_objs import Table
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot

In [3]:
def print_head(f, size=1):
    with open(f, 'r') as read_file:
        print(read_file.readline())

In [4]:
print_head('review.json')

{"review_id":"VfBHSwC5Vz_pbFluy07i9Q","user_id":"cjpdDjZyprfyDG3RlkVG3w","business_id":"uYHaNptLzDLoV_JZ_MuzUA","stars":5,"date":"2016-07-12","text":"My girlfriend and I stayed here for 3 nights and loved it. The location of this hotel and very decent price makes this an amazing deal. When you walk out the front door Scott Monument and Princes street are right in front of you, Edinburgh Castle and the Royal Mile is a 2 minute walk via a close right around the corner, and there are so many hidden gems nearby including Calton Hill and the newly opened Arches that made this location incredible.\n\nThe hotel itself was also very nice with a reasonably priced bar, very considerate staff, and small but comfortable rooms with excellent bathrooms and showers. Only two minor complaints are no telephones in room for room service (not a huge deal for us) and no AC in the room, but they have huge windows which can be fully opened. The staff were incredible though, letting us borrow umbrellas for t

In [5]:
print_head('business.json')

{"business_id": "YDf95gJZaq05wvo7hTQbbQ", "name": "Richmond Town Square", "neighborhood": "", "address": "691 Richmond Rd", "city": "Richmond Heights", "state": "OH", "postal_code": "44143", "latitude": 41.5417162, "longitude": -81.4931165, "stars": 2.0, "review_count": 17, "is_open": 1, "attributes": {"RestaurantsPriceRange2": 2, "BusinessParking": {"garage": false, "street": false, "validated": false, "lot": true, "valet": false}, "BikeParking": true, "WheelchairAccessible": true}, "categories": ["Shopping", "Shopping Centers"], "hours": {"Monday": "10:00-21:00", "Tuesday": "10:00-21:00", "Friday": "10:00-21:00", "Wednesday": "10:00-21:00", "Thursday": "10:00-21:00", "Sunday": "11:00-18:00", "Saturday": "10:00-21:00"}}



In [6]:
def get_file_size(f):
    return sum(1 for _ in open(f))

In [7]:
print('Total number of businesses:', get_file_size('business.json'))
print('Total number of reviews', get_file_size('review.json'))

Total number of businesses: 156639
Total number of reviews 4736897


## Load Data 

In [8]:
def extract_data(f):
    with open(f, 'r') as read_file:
        for _ in read_file:
            yield json.loads(_)

In [9]:
business_df = pd.io.json.json_normalize(list(extract_data('business.json')))

In [10]:
business_df.shape

(156639, 101)

In [11]:
#edit city
def audit_cities(row, city_map):
    for k in city_map:
        if k in row:
            return row.replace(k, city_map[k])
    return row

In [12]:
city_map = json.loads(open('businessEdits.json', 'r').read())
business_df['city'] = business_df['city'].str.lower().str.strip()
business_df['city'] = business_df['city']\
                        .apply(lambda x: audit_cities(x, city_map))
business_df['state'] = business_df['state'].str.upper().str.strip()
#handle pittsburgh misspelling
business_df.loc[business_df.city == 'pittsburghh', 'city'] = 'pittsburgh'

## State and City Data 

In [13]:
stat_table = Table(
    header = {
        'values': ['Total State/Province Count', 'Total City Count', 'Avg # of Reviews']
    }, 
    cells = {
        'values': [
            [len(business_df.state.unique())], 
            [len(business_df.city.unique())], 
            [round(business_df.review_count.mean(), 0)]
        ], 
        'align': 'middle'
    }
)
iplot([stat_table])

In [14]:
val_df = business_df.state.value_counts().reset_index()
data = [
    {
        'type': 'choropleth',
        'colorscale': [[0, 'rgb(222,235,247)'], [1, 'rgb(8,81,156)']],
        'autocolorscale': False, 
        'locations': val_df['index'].astype(str), 
        'z': val_df.state.astype(float),
        'locationmode': 'USA-states',
        'text': val_df.state,
        'marker': {
            'line': {
                'color': 'rgb(255, 255, 255)',
                'width': 2
            }
        },
        'colorbar': {
            'title': 'Review Count'
        }
    }
]
layout = {
    'title': 'Location of USA Businesses in Dataset', 
    'geo': {
        'scope': 'usa',
        'projection': {
            'type': 'albers usa',
        }
    }
}
fig = {'data': data, 'layout': layout}
iplot(fig)

In [15]:
#top 10 state and city counts
top10_state_counts = business_df.state.value_counts()[:10]
top10_city_counts = business_df.city.value_counts()[:10]
figs = [
    top10_state_counts.iplot(kind='bar', 
                             asFigure=True, 
                             xTitle='State', 
                             yTitle='Count', 
                             color='blue'), 
    top10_city_counts.iplot(kind='bar', 
                            asFigure=True, 
                            xTitle='City', 
                            yTitle='Business Count', 
                            color='blue')
]
subplots = cf.subplots(figs, subplot_titles=['Top 10 States by Business Count', 
                                             'Top 10 Cities by Business Count'])
cf.iplot(subplots, legend=False)

In [16]:
#top 5 cities by business count in the top 3 states
top5_statecity = business_df[['city', 'state', 'address']]\
                            .loc[business_df.state.isin(('AZ', 'NV', 'ON'))]\
                            .groupby(['city','state'], as_index=False).address\
                            .count()
top5_statecity = top5_statecity.groupby(['state'], group_keys=False)\
                               .apply(lambda x: x.nlargest(5, 'address'))
figs = [
    top5_statecity.loc[top5_statecity.state==s, ('city', 'address')]\
                  .iplot(kind='bar', 
                         x='city', 
                         y='address', 
                         asFigure=True, 
                         color=next(colorscale))
    for s in top5_statecity.state.unique()]
cf.iplot(cf.subplots(figs, 
                     subplot_titles=['Arizona', 'Nevada', 'Ontario'], 
                     shape=(3, 1), 
                     vertical_spacing=0.25),  
         legend=False)

In [17]:
#avg amount of time each place is open by state
hours = business_df.ix[:, list(filter(lambda x: 'hours' in x, business_df.columns))]
hours.fillna(0, inplace=True)

/home/blake/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated



In [18]:
hours.head()

,hours.Friday,hours.Monday,hours.Saturday,hours.Sunday,hours.Thursday,hours.Tuesday,hours.Wednesday
0,10:00-21:00,10:00-21:00,10:00-21:00,11:00-18:00,10:00-21:00,10:00-21:00,10:00-21:00
1,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00
2,10:00-19:00,10:00-19:00,10:00-18:00,12:00-17:00,10:00-19:00,10:00-19:00,10:00-19:00
3,9:00-17:00,9:00-17:00,0,0,9:00-17:00,9:00-17:00,9:00-17:00
4,0,0,0,0,0,0,0


In [19]:
from datetime import datetime

In [20]:
def find_time_length(row):
    if row:
        str_row = str(row)
        time_format = '%H:%M'
        split_row = str_row.split('-')
        time_difference = datetime.strptime(split_row[-1], time_format)\
                        - datetime.strptime(split_row[0], time_format)
        return time_difference.seconds // 3600

In [21]:
hours_open_df = hours.applymap(find_time_length)
hours_open_df['state'] = business_df.state
hours_open_df.columns = [c.replace('hours.', '') for c in hours_open_df.columns]

In [22]:
melted_hours = hours_open_df.melt(id_vars='state', 
                                  value_vars=list(filter(lambda x: x != 'state', hours_open_df.columns)), 
                                  value_name='hours', 
                                  var_name='day')
melted_hours_grouped = melted_hours.groupby('state', as_index=False).hours.mean()
melted_hours_grouped.sort_values(by='hours', ascending=False, inplace=True)
#get rid of states with numbers in them
melted_hours_grouped = melted_hours_grouped[~melted_hours_grouped.state.str.contains('\d+')]

In [23]:
iplot(melted_hours_grouped[:20].iplot(kind='bar', 
                                      x='state', \
                                      y='hours', \
                                      color='blue', 
                                      title='Average # of Hours Open by State',
                                      xTitle='State', 
                                      yTitle='Avg. # of Hours',
                                      asFigure=True))

### Business Categories 

In [24]:
category_df = pd.concat([business_df[['city', 'state']], 
                         business_df.categories.apply(lambda x: ','.join(x))], 
                         axis=1)
category_df = pd.concat([category_df, category_df.categories.str.split(',', expand=True)], axis=1)
category_df.drop('categories', axis=1, inplace=True)
melted_category_df = category_df.melt(id_vars=['city', 'state'], 
                                      value_vars = list(filter(lambda x: x not in ['city', 'state'], 
                                                               category_df.columns)), 
                                      value_name='category')
melted_category_df.drop('variable', axis=1, inplace=True)
melted_category_df = melted_category_df[~melted_category_df.category.isnull()]

In [25]:
#top 20 categories
f = melted_category_df.category.value_counts(ascending=False)[:20].sort_values()\
                               .iplot(kind='barh', 
                                      color='blue', 
                                      title='Top Categories by Business Count', 
                                      xTitle='Business Count',
                                      asFigure=True)
f['layout']['yaxis1']['tickfont']['size'] = 9
iplot(f)

### Star Ratings

### Review Counts 

In [26]:
review_counts = pd.DataFrame({'Review Count': [r.get('review_count') for r in extract_data('business.json')]})

In [27]:
cut_review_outliers = review_counts[review_counts['Review Count'] < review_counts['Review Count'].quantile(.95)]
iplot(cut_review_outliers.iplot(kind='histogram', 
                                asFigure=True, 
                                color='blue', 
                                title='Review Count Frequency', 
                                xTitle='Review Count', 
                                yTitle='Frequency'))

In [28]:
#most reviewed spot in dataset
top_reviewed_biz = business_df.iloc[business_df.review_count.idxmax()].to_dict()
print('Top Business Name:', top_reviewed_biz.get('name'))
print('Top Reviewed Business:', ', '.join([top_reviewed_biz.get('address'), 
                                            top_reviewed_biz.get('city').title(), 
                                            top_reviewed_biz.get('state')]))
print('# of Reviews:', top_reviewed_biz.get('review_count'))
print('# of Stars:', top_reviewed_biz.get('stars'))
'Link: ', display(HTML('<a href="https://www.yelp.com/biz/mon-ami-gabi-las-vegas-2">Link to page</a>'));

Top Business Name: Mon Ami Gabi
Top Reviewed Business: 3655 Las Vegas Blvd S, Las Vegas, NV
# of Reviews: 6979
# of Stars: 4.0
